In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

2023-08-23 10:14:17.040196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# load data locally
images = np.load("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/data/cleaned/5000_images.npy")
images = images.astype(np.float32)
images = images/255
prices = np.load("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/data/cleaned/5000_prices.npy")

data = tf.data.Dataset.from_tensor_slices((images, prices))

In [25]:
import boto3
import io
import pickle

# load data from s3
s3_client = boto3.client('s3')

img_df_data = io.BytesIO()
s3_client.download_fileobj('sneaker-dataset', '5000_images.pkl', img_df_data)
img_df_data.seek(0)
images = pickle.load(img_df_data)
images = np.stack(images)
images = images.astype(np.float32)
images = images/255

prices_df_data = io.BytesIO()
s3_client.download_fileobj('sneaker-dataset', '5000_prices.pkl', prices_df_data)
prices_df_data.seek(0)
prices = pickle.load(prices_df_data)

data = tf.data.Dataset.from_tensor_slices((images, prices))


In [ ]:
# split data
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train = train.cache()
train = train.shuffle(train_size)
train = train.batch(80)
train = train.prefetch(80)

val = val.cache()
val = val.shuffle(val_size)
val = val.batch(80)
val = val.prefetch(80)


In [ ]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation = 'linear'))


In [ ]:
model.compile('adam', tf.losses.MeanSquaredError())

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train, epochs=40, validation_data=val)

In [ ]:
model.save("/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/models/model_1.h5")